In [1]:
%%capture
!pip install pyserini==0.10.0.1

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
import pandas as pd
import numpy as np

In [3]:
queries_train=pd.read_table('../datasets/queries.doctrain.tsv',header=None)
queries_train.columns=['qid','query']

#long queries
queries_train['len']=queries_train['query'].apply(lambda x: 1 if len(x) > 110 else np.nan)

#short queries
#queries_train['len']=queries_train['query'].apply(lambda x: 1 if len(x) > 25 and len(x) < 35 else np.nan)

queries_train=queries_train.dropna()
queries=queries_train.reset_index(drop=True).tail(400)

print('Shape=>',queries.shape)
print(queries.head())

Shape=> (400, 3)
       qid                                              query  len
64  402449  is an appliance such as a leg brace that is sp...  1.0
65  662128  what force do scientists think was responsible...  1.0
66  683829  what is a form of fiction that draws imaginati...  1.0
67  683830  what is a form of government in which power is...  1.0
68  392797  immune surveillance is a process in which ____...  1.0


In [4]:
#queries = queries.drop(['len'], axis=1)
queries.head()

,qid,query,len
64,402449,is an appliance such as a leg brace that is sp...,1.0
65,662128,what force do scientists think was responsible...,1.0
66,683829,what is a form of fiction that draws imaginati...,1.0
67,683830,what is a form of government in which power is...,1.0
68,392797,immune surveillance is a process in which ____...,1.0


In [5]:
topics = {}
for index, row in queries.iterrows():
    topics[row['qid']] = {'title': str(row['query'])}

len(topics)

400

In [35]:
import json

with open('queries.txt', 'w') as fp:
    df = queries.to_json()
    json.dump(df, fp)

In [6]:
from pyserini.search import SimpleSearcher

def run_all_queries(file, topics, searcher):
    with open(file, 'w') as runfile:
        cnt = 0
        print('Running {} queries in total'.format(len(topics)))
        for id in topics:
            query = topics[id]['title']
            hits = searcher.search(query, 1000)
            for i in range(0, len(hits)):
                _ = runfile.write('{} Q0 {} {} {:.6f} Anserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
            cnt += 1
            if cnt % 100 == 0:
                print(f'{cnt} queries completed')

searcher = SimpleSearcher.from_prebuilt_index('msmarco-doc')

run_all_queries('run-msmarco-doc.txt', topics, searcher)

RuntimeError: 
        Unable to find libjvm.so, (tried ['/usr/lib/jvm/java-11-openjdk-amd64/jre/lib/jli/libjli.dylib', '/usr/lib/jvm/java-11-openjdk-amd64/lib/jli/libjli.dylib', '/usr/lib/jvm/java-11-openjdk-amd64/lib/libjli.dylib'])
        you can use the JVM_PATH env variable with the relative path
        from JAVA_HOME to libjvm.so to override this lookup, if you know
        where pyjnius should look for it.

        e.g:
            export JAVA_HOME=/usr/lib/jvm/java-8-oracle/
            export JVM_PATH=./jre/lib/amd64/server/libjvm.so
            # run your program
        

In [30]:
%%capture
!wget -O jtreceval-0.0.5-jar-with-dependencies.jar https://search.maven.org/remotecontent?filepath=uk/ac/gla/dcs/terrierteam/jtreceval/0.0.5/jtreceval-0.0.5-jar-with-dependencies.jar

In [36]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar msmarco-doctrain-top100 run-msmarco-doc.txt

trec_eval.get_qrels: Malformed line 1
trec_eval: Quit in file 'queries.txt'


In [37]:
!ls

jtreceval-0.0.5-jar-with-dependencies.jar  queries.txt
queries.doctrain.tsv			   run-msmarco-doc.txt
queries.doctrain.txt			   sample_data
